In [258]:
from nltk.corpus import wordnet as wn
from pprint import pprint
import nltk
from nltk.corpus import brown

## Defining the State Machine Graph

In [259]:
DAG=[
    [1,1,0,0,0,0,0],
    [0,1,1,1,1,1,0],
    [0,1,1,0,0,1,1],
    [0,1,1,1,1,1,1],
    [0,1,0,0,1,1,1],
    [0,0,0,0,0,1,1],
    [1,0,0,0,0,0,0]
]
pprint(DAG)

[[1, 1, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 1, 1],
 [0, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 1, 1],
 [1, 0, 0, 0, 0, 0, 0]]


## Preprocess The Sentence to remove Articles, Prepositions 

In [281]:
preposition=['in','on','at','under','of','off']

sentence='Bus Driver pulls off incredible move'

#Extract words from sentence
words=sentence.split(' ')

new_word=[]
for i in words:
    if i=='The'or i=='a' or i=='the' or i=='A'or i=='An' or i=='an' or i in preposition:
        new_word=new_word
    else:
        new_word.append(i)
pprint(new_word)

['Bus', 'Driver', 'pulls', 'incredible', 'move']


## GetIndex for patricular type of Word

In [282]:
def get_index_for_word(word):
    if word=='verb':
        return 1
    elif word=='adjective':
        return 3
    elif word=='noun':
        return 4
    elif word=='adverb':
        return 5
    elif word=='conjunction':
        return 6
    else:
        return 1

## Define data variables

In [283]:
noun_ind=['NN','NR']
verb_ind=['VBG','BEZ']
adj_in=['JJ']
pos_word=['good','beautiful','awesome']
neg_word=['bad','worst','ruthless','forget']


In [284]:
def get_type(word):
    table = nltk.FreqDist(t for w, t in brown.tagged_words() if w.lower() == word.lower())
    k=table.most_common()[0][0]
    #print(word+str(':')+str(k))
    if k in noun_ind:
        return 'noun'
    elif k in verb_ind:
        return 'verb'
    elif k in adj_in:
        return 'adjective'

In [285]:
def get_polarity(word):
    if word.lower() in neg_word:
        return -1
    else:
        return 1

## Feed sentence into State machine

In [286]:
prev_pol=1
mypol=1
ind_pol=1
polarity_final=1

first_index=1
index_new_word=0
while new_word[index_new_word]=='not':
    index_new_word+=1
    ind_pol*=-1

#Get type of word for first word
table = nltk.FreqDist(t for w, t in brown.tagged_words() if w.lower() == new_word[index_new_word].lower())
k=table.most_common()[0][0]

print(ind_pol)
if k in noun_ind:
    first_index=first_index
elif k in adj_in:
    first_index=get_index_for_word('adjective')
    prev_pol=ind_pol*get_polarity(new_word[index_new_word])
    index_new_word+=1
    ind_pol=1
    mypol=1
print(prev_pol)
print(first_index)

1
1
1


In [287]:

for i in range(index_new_word,len(new_word)):
    type_word=get_type(new_word[i])
    z=get_index_for_word(type_word)
    if new_word[i]== 'not':
        ind_pol=-1
    else:
        mypol=get_polarity(new_word[i])
        if DAG[first_index][z]==1:
            first_index=z
            temp_pol=ind_pol*mypol
            if type_word!='adjective':
                fin_pol=prev_pol*temp_pol
            else:
                fin_pol=mypol*ind_pol
            prev_pol=fin_pol
            mypol=1
            ind_pol=1
            
        else:
            print('cannot determine')
            break
    print(new_word[i]+': '+str(prev_pol))
        
        
if prev_pol==-1:
    print('negative')
else:
    print('positive')


Bus: 1
Driver: 1
pulls: 1
incredible: 1
move: 1
positive
